# Google Calendar Events Downloader

* Download all of your google calendar events history and export into csv. 
* For simple data analysis and visualization, see: [google_calendar_data_analysis](https://github.com/markwk/qs_ledger/tree/master/google_calendar/google_calendar_data_analysis.ipynb)

----

## Google Calendar API Docs

* https://developers.google.com/calendar/v3/reference/  
* https://developers.google.com/resources/api-libraries/documentation/calendar/v3/python/latest/

------

# Initial Install and Setup

Follow Steps here: https://developers.google.com/calendar/quickstart/python

1. Click Button "ENABLE THE GOOGLE CALENDAR API"
2. Select + Create a new project.
3. Download the configuration file.
4. Move the downloaded file to your working directory and ensure it is named credentials.json.
5. In terminal or command line, navigate to your project's working directory
6. Run `python quickstart.py`

Afterwards:

> The sample will attempt to open a new window or tab in your default browser. If this fails, copy the URL from the console and manually open it in your browser.

> If you are not already logged into your Google account, you will be prompted to log in. If you are logged into multiple Google accounts, you will be asked to select one account to use for the authorization.

> Click the Accept button.
> The sample will proceed automatically, and you may close the window/tab.

-----

## Libraries and Dependencies

In [1]:
!pip install google-api-python-client
!pip install oauth2client
!pip install gspread oauth2client

In [2]:
!pip install google-api-python-client
!pip install oauth2client
!pip install gspread oauth2client

In [3]:


from __future__ import print_function
import datetime
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time


----

## Authorization Check

In [4]:
# If modifying these scopes, delete the file token.json.
SCOPES = 'https://www.googleapis.com/auth/calendar.readonly'

In [5]:
def authorize():
    store = file.Storage('token.json')
    creds = store.get()
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
        creds = tools.run_flow(flow, store)
    global service
    service = build('calendar', 'v3', http=creds.authorize(Http()))

In [6]:
authorize()

In [7]:
scopesGoogleSheets = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

In [8]:
#Authorize google sheets
creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scopesGoogleSheets)

client = gspread.authorize(creds)


    

----

## Quick Start Test in Jupyter

In [9]:
def test():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """

    authorize()
    
    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId='primary', timeMin=now,
                                        maxResults=10, singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(start, event['summary'])

In [10]:
test()

Getting the upcoming 10 events
2020-09-27T15:45:00+02:00 Shower
2020-09-28T15:45:00+02:00 Shower
2020-09-29T15:45:00+02:00 Shower
2020-09-30T15:45:00+02:00 Shower
2020-10-01T15:45:00+02:00 Shower
2020-10-02T15:45:00+02:00 Shower
2020-10-03T15:45:00+02:00 Shower
2020-10-04T15:45:00+02:00 Shower
2020-10-05T15:45:00+02:00 Shower
2020-10-06T15:45:00+02:00 Shower


----

## Get List User Calendars

In [11]:
calendars_list = service.calendarList().list().execute()
calendars_dict = calendars_list.get('items', [])

if not calendars_dict:
    print('No calendars found.')

print("Printing Calendar Names...")

calendar_ids = []
calenderSummaries = []

for calendar in calendars_dict:
    print(calendar['id'])
    print(calendar['summary'])
    # create a list of calendar ids
    calendar_ids.append(calendar['id'])
    calenderSummaries.append(calendar['summary'])

Printing Calendar Names...
xxguitarsenxx@gmail.com
xxguitarsenxx@gmail.com
nn6uignfh2hvep45ecvkbc9htg@group.calendar.google.com
Music / Guitar
n2tm4rta6mkhich6svstnt4bg0@group.calendar.google.com
Other work
f071lc3i400hesulskq2tbmb08@group.calendar.google.com
Free time
bm96o9ngs4e1l5j543qiquhl94@group.calendar.google.com
Sleep
r0r1149ktiu28a03qfg2e76ud0@group.calendar.google.com
Study
hj8iu2n1vl4mjdekp09phsfgq8@group.calendar.google.com
Eat
ijv28ivour5u668d0m6eic0c8g@group.calendar.google.com
P4
jd6vequjbcg3qar3iis7s0ctl5vjjrdm@import.calendar.google.com
https://www.moodle.aau.dk/calendar/export_execute.php?userid=76137&authtoken=5dd2d3fd598d609c8b062cfb15d2412c3ec95217&preset_what=all&preset_time=custom
hkjg98uhl7l2oa941au5i7u47o@group.calendar.google.com
P5 - AStep
323o03ja1q6hdm3u7m232e715g@group.calendar.google.com
Self improvement
coic30en3lkqbpe48lqo574n1s@group.calendar.google.com
Rusperioden 2020
8u8om17augs6reg5ndeurqhjn8@group.calendar.google.com
Social
c25c4tqubvr1lp9v28fbbh

In [12]:
print("You have " + str(len(calendar_ids)) + " calendars in this account's Google Calendar")

You have 21 calendars in this account's Google Calendar


In [13]:
# dataframing calendars data
calendars = pd.DataFrame.from_dict(calendars_dict)

In [14]:
calendars.to_csv('data/calendars_list.csv')

----

## Get Past Events from Calendar

In [15]:
def get_calendar_events(calendar_id):
    beginning_time = "2004-01-01T00:00:00.356167Z"
    now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time

    page_count = 0
    events_list = []

    print('Getting all of the past events')

    events = service.events()
    request = events.list(
        #calendarId = 'primary',
        calendarId = calendar_id,
        timeMin=beginning_time,
        timeMax=now,
        #maxResults=2500,
        maxResults=100000,
        singleEvents=True,
        # showDeleted=True,
        orderBy='updated')

    while request is not None:
        
      page_count = page_count + 1
      print("Getting " + str(page_count) + " page of events...")
      eventsResult = request.execute()

      # Do something with the activities
      print(len(eventsResult.get('items', [])))
      events_list = events_list + list(eventsResult.get('items', []))

      request = events.list_next(request, eventsResult)
    
    return events_list

In [16]:
#Give calendars correct names
def CalendarName(name):
    if name == "5hbqqk1mhmiiog9rul08cg9quo@group.calendar.google.com":
        return "Deep work"
    elif name == "8u8om17augs6reg5ndeurqhjn8@group.calendar.google.com":
        return "Social"
    elif name == "9hq0ge3o96lcu3v7fhene9cv30@group.calendar.google.com":
        return "Training"
    elif name == "323o03ja1q6hdm3u7m232e715g@group.calendar.google.com":
        return "Self improvement"
    elif name == "addressbook#contacts@group.v.calendar.google.com":
        return "Address book"
    elif name == "bm96o9ngs4e1l5j543qiquhl94@group.calendar.google.com":
        return "Sleep"
    elif name == "c25c4tqubvr1lp9v28fbbhs7m4@group.calendar.google.com":
        return "Transport"
    elif name == "coic30en3lkqbpe48lqo574n1s@group.calendar.google.com":
        return "Tutor?"
    elif name == "da.danish#holiday@group.v.calendar.google.com":
        return "Danish holidays"
    elif name == "e_2_da#weeknum@group.v.calendar.google.com":
        return "Week numbers"
    elif name == "f071lc3i400hesulskq2tbmb08@group.calendar.google.com":
        return "Free time"
    elif name == "hf35h1qvf1vhpt0t9ds93mame8@group.calendar.google.com":
        return "Deadlines/Reminders"
    elif name == "hj8iu2n1vl4mjdekp09phsfgq8@group.calendar.google.com":
        return "Eating"
    elif name == "ijv28ivour5u668d0m6eic0c8g@group.calendar.google.com":
        return "P4"
    elif name == "jd6vequjbcg3qar3iis7s0ctl5vjjrdm@import.calendar.google.com":
        return "Moodle"
    elif name == "larsnyborgpedersen@gmail.com":
        return "Other"
    elif name == "n2tm4rta6mkhich6svstnt4bg0@group.calendar.google.com":
        return "Other work"
    elif name == "nn6uignfh2hvep45ecvkbc9htg@group.calendar.google.com":
        return "Guitar"
    elif name == "r0r1149ktiu28a03qfg2e76ud0@group.calendar.google.com":
        return "Study"
    elif name == "xxguitarsenxx@gmail.com":
        return "xxguitarsenxx"
    else:
        return "Didn't find a calendar!"

In [22]:
index = 0
event_df = []

sheet = client.open("Google Calendar").sheet1  # Open the spreadhseet

# generate csv of all calendars in google calendar
for calender_id in calendar_ids:
    print("calender_id is: " + calender_id)
    #if calender_id != "xxguitarsenxx@gmail.com" and calender_id != "nn6uignfh2hvep45ecvkbc9htg@group.calendar.google.com" and calender_id != "n2tm4rta6mkhich6svstnt4bg0@group.calendar.google.com" and calender_id != "f071lc3i400hesulskq2tbmb08@group.calendar.google.com" and calender_id != "bm96o9ngs4e1l5j543qiquhl94@group.calendar.google.com" and calender_id != "r0r1149ktiu28a03qfg2e76ud0@group.calendar.google.com" and calender_id != "hj8iu2n1vl4mjdekp09phsfgq8@group.calendar.google.com" and calender_id != "ijv28ivour5u668d0m6eic0c8g@group.calendar.google.com" and calender_id != "jd6vequjbcg3qar3iis7s0ctl5vjjrdm@import.calendar.google.com" and calender_id != "323o03ja1q6hdm3u7m232e715g@group.calendar.google.com" and calender_id != "coic30en3lkqbpe48lqo574n1s@group.calendar.google.com" and calender_id != "8u8om17augs6reg5ndeurqhjn8@group.calendar.google.com" and calender_id != "c25c4tqubvr1lp9v28fbbhs7m4@group.calendar.google.com" and calender_id != "5hbqqk1mhmiiog9rul08cg9quo@group.calendar.google.com" and calender_id != "9hq0ge3o96lcu3v7fhene9cv30@group.calendar.google.com" and calender_id != "larsnyborgpedersen@gmail.com" and calender_id != "hf35h1qvf1vhpt0t9ds93mame8@group.calendar.google.com": 

    #Get events
    events_list = get_calendar_events(calender_id)
    event_df = pd.DataFrame(events_list)



    #Make into .CSV file
    filename = str(calender_id) + ".csv"
    filepath = 'data/' + filename
    event_df.to_csv(filepath, index=False)
    print(calender_id)    




    numRows = len(sheet.col_values(1))  # Get the number of rows with values in the sheet

    print("Number of rows is: " + str(numRows) + "!!!!!!!!!!!!!!!!!!!!!!!!")

    for eventIndex, row in event_df.iterrows():   
        #Extract values
        summary = row[7]

        #For some reason some of the calendars have more attributes / variables.
        if (calender_id == "larsnyborgpedersen@gmail.com"):
            startString = list(row[12].values())[0]
            endString = list(row[13].values())[0]
        elif(calender_id == "addressbook#contacts@group.v.calendar.google.com") or (calender_id == "hkjg98uhl7l2oa941au5i7u47o@group.calendar.google.com") or (calender_id == "jd6vequjbcg3qar3iis7s0ctl5vjjrdm@import.calendar.google.com"):
            startString = list(row[11].values())[0]
            endString = list(row[12].values())[0]
        else:
            startString = list(row[10].values())[0]
            endString = list(row[11].values())[0]

        start = pd.to_datetime(startString)
        end = pd.to_datetime(endString)

        duration = end - start
        durationMinutes = duration.total_seconds() / 60
        durationMinutes
        
        #Only transfer if after this date
        startDate = pd.to_datetime(startString)
        startDate = startDate.tz_localize(None)
        afterThisDate = pd.to_datetime("15-08-2020")
        
        if startDate >= afterThisDate:
            sheet.insert_row ([calenderSummaries[calendar_ids.index(calender_id)],summary, startString, endString, durationMinutes], numRows + 1) 
            numRows = numRows + 1
            time.sleep(2.2)


    index +=1


calender_id is: xxguitarsenxx@gmail.com
Getting all of the past events
Getting 1 page of events...
833
xxguitarsenxx@gmail.com
Number of rows is: 5564!!!!!!!!!!!!!!!!!!!!!!!!
calender_id is: nn6uignfh2hvep45ecvkbc9htg@group.calendar.google.com
Getting all of the past events
Getting 1 page of events...
145
nn6uignfh2hvep45ecvkbc9htg@group.calendar.google.com
Number of rows is: 5564!!!!!!!!!!!!!!!!!!!!!!!!
calender_id is: n2tm4rta6mkhich6svstnt4bg0@group.calendar.google.com
Getting all of the past events
Getting 1 page of events...
233
n2tm4rta6mkhich6svstnt4bg0@group.calendar.google.com
Number of rows is: 5607!!!!!!!!!!!!!!!!!!!!!!!!
calender_id is: f071lc3i400hesulskq2tbmb08@group.calendar.google.com
Getting all of the past events
Getting 1 page of events...
426
f071lc3i400hesulskq2tbmb08@group.calendar.google.com
Number of rows is: 5675!!!!!!!!!!!!!!!!!!!!!!!!
calender_id is: bm96o9ngs4e1l5j543qiquhl94@group.calendar.google.com
Getting all of the past events
Getting 1 page of events..

PermissionError: [Errno 13] Permission denied: 'data/hkjg98uhl7l2oa941au5i7u47o@group.calendar.google.com.csv'

In [ ]:
CalenderIDs

In [ ]:
### how to
#row = sheet.row_values(3)  # Get a specific row
#col = sheet.col_values(3)  # Get a specific column
#cell = sheet.cell(1,2).value  # Get the value of a specific cell
#sheet.update_cell(2,2, "CHANGED")  # Update one cell
#data = sheet.get_all_records()  # Get a list of all records
#insertRow = ["lars", 5, "red", "blue"]
    
    

        



In [ ]:
event_df

In [ ]:

events_list
